## <font color='green'> <div align="center">In the name of God </div></font>

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>

# Showing that when S2='Sw', then the 'svd' solver does not solve the generalized eigenvalue problem

In [1]:
import numpy as np

## Constructing a dataset with 2 classes, 4 samples, and 4 dimensions

In [2]:
C = 2
D  = 4
N = 4
N1 = 2
N2 = 2

In [3]:
X = [[-2, -1, -1, -1],
     [-1,  1, -1,  1],
     [-1,  1,  1,  -1],          
     [1,  1,  -1, -2]]
X = np.array (X, dtype = float)
y = np.array ([1,1,2,2])

In [4]:
mu_1 = np.mean (X[0:2,:], axis = 0)
mu_2 = np.mean (X[2:4,:], axis = 0)
mu   = np.mean (X, axis = 0)
print (mu, mu_1, mu_2)

[-0.75  0.5  -0.5  -0.75] [-1.5  0.  -1.   0. ] [ 0.   1.   0.  -1.5]


In [5]:
def computeSb(X, Ni, C):
    N, dim = X.shape
    Sb = 0
    M = np.mean(X, axis=0)
    idx1 = 0
    for c in range (C):
        idx2 = idx1 + Ni[c]
        Mc = np.mean(X[idx1:idx2,:], axis = 0) - M
        Sb = Sb + Ni[c] / N * np.outer (Mc, Mc)
    return Sb

In [6]:
def computeSw(X, Ni, C):
    N, dim = X.shape
    Sw = 0
    Xnew = X.copy()
    idx1 = 0
    for c in range (C):
            idx2 = idx1 + Ni[c]
            Xnew[idx1:idx2,:] -= np.mean(Xnew[idx1:idx2,:], axis = 0)
            idx1 = idx2
    Sw = Xnew.T @ Xnew  / N
    return Sw

In [7]:
def computeSt(X):
    N, dim = X.shape
    M = np.mean(X, axis=0)
    X = X - M
    St = 1 / N * X.T @ X
    return St

# <font color='red'> First we run the SVD algorithm with $S_1=S_b$ and $S_2=S_w$</font>

## 1: Compute $H_1$ and $H_2$

In [8]:
Hb = np.array([np.sqrt(N1/N) * (mu_1-mu), np.sqrt(N2/N) * (mu_2-mu)])
Hw = 1/np.sqrt(N) * np.array([X[0,:]- mu_1, X[1,:]- mu_1, X[2,:]- mu_2, X[3,:]- mu_2])
print ('Hb=\n', Hb)
print ('Hw=\n', Hw)

Hb=
 [[-0.53033009 -0.35355339 -0.35355339  0.53033009]
 [ 0.53033009  0.35355339  0.35355339 -0.53033009]]
Hw=
 [[-0.25 -0.5   0.   -0.5 ]
 [ 0.25  0.5   0.    0.5 ]
 [-0.5   0.    0.5   0.25]
 [ 0.5   0.   -0.5  -0.25]]


## Checking that $H_1$ and $H_2$ are computed correctly

In [9]:
Ni = np.array ([2,2], dtype=int)
Sb = computeSb(X, Ni, C)
Sw = computeSw(X, Ni, C)
print (np.linalg.norm (Hw.T @ Hw - Sw ))
print (np.linalg.norm (Hb.T @ Hb - Sb ))

0.0
2.9373740229761033e-16


## 2: Compute the reduced SVD of $H_2$ to obtain $H_2=U\Sigma V^T$

In [10]:
U, Sigma, VH = np.linalg.svd(Hw, full_matrices=False)
idx = Sigma > 10e-6
SigmaInv = np.diag (1/Sigma[idx])
Sigma = np.diag (Sigma[idx])
V = VH.T
V = V[:,idx]

## Checking that $V$ is computed correctly

In [11]:
print (np.linalg.norm(V @ Sigma @ Sigma @ V.T - Sw))

5.574188198562492e-16




## 3: Compute $Y=H_1 V \Sigma^{-1}$

In [12]:
Y = Hb @ V @ SigmaInv
print (Y)

[[ 0.0745356 -0.2732972]
 [-0.0745356  0.2732972]]


## 4: Compute reduced SVD of $Y$ to obtain $Y=\tilde{U}\tilde{\Sigma}\tilde{V}^T$

In [13]:
U_tilde, Sigma_tilde, VH_tilde = np.linalg.svd(Y, full_matrices=False)
idx = Sigma_tilde > 10e-6
Sigma_tildeInv = np.diag (1/Sigma_tilde[idx])
Sigma_tilde = np.diag (Sigma_tilde[idx])
V_tilde = VH_tilde.T
V_tilde = V_tilde[:,idx]
print (V_tilde)

[[-0.26311741]
 [ 0.96476382]]


## 5: $A=V\Sigma^{-1}\tilde{V}$

In [14]:
A_w = V @ SigmaInv @ V_tilde
print (A_w)

[[ 0.67796691]
 [ 0.12326671]
 [-0.61633355]
 [-0.18490007]]


## 6: Computing objective function for the solution $A$ when $S_2=S_w$

In [15]:
Sb = Hb.T @ Hb
Sw = Hw.T @ Hw
St = Sw + Sb
StY = A_w.T @ St @ A_w
SbY = A_w.T @ Sb @ A_w
obj_sw = np.trace(np.linalg.pinv (StY) @ SbY)
print ('Objective Value = ', obj_sw)

Objective Value =  0.13829787234042565


# <font color='red'> Now we run the SVD algorithm with $S_1=S_b$ and $S_2=S_t$</font>

## 1: Compute $H_1$ and $H_2$

In [16]:
Hb = np.array([np.sqrt(N1/N) * (mu_1-mu), np.sqrt(N2/N) * (mu_2-mu)])
Ht = 1/np.sqrt(N) * (X-mu)
print ('Hb=\n', Hb)
print ('Ht=\n', Hw)

Hb=
 [[-0.53033009 -0.35355339 -0.35355339  0.53033009]
 [ 0.53033009  0.35355339  0.35355339 -0.53033009]]
Ht=
 [[-0.25 -0.5   0.   -0.5 ]
 [ 0.25  0.5   0.    0.5 ]
 [-0.5   0.    0.5   0.25]
 [ 0.5   0.   -0.5  -0.25]]


## Checking that $H_1$ and $H_2$ are computed correctly

In [17]:
print (np.linalg.norm (Ht.T @ Ht - St ))

2.220446049250313e-16


## 2: Compute the reduced SVD of $H_2$ to obtain $H_2=U\Sigma V^T$

In [18]:
U, Sigma, VH = np.linalg.svd(Ht, full_matrices=False)
idx = Sigma > 10e-6
SigmaInv = np.diag (1/Sigma[idx])
Sigma = np.diag (Sigma[idx])
V = VH.T
V = V[:,idx]

## Checking that $V$ is computed correctly

In [19]:
print (np.linalg.norm(V @ Sigma @ Sigma @ V.T - St))

1.5279438074046465e-15


## 3: Compute $Y=H_1 V \Sigma^{-1}$

In [20]:
Y = Hb @ V @ SigmaInv
print (Y)

[[-6.09317497e-01 -1.57009246e-16  3.58792680e-01]
 [ 6.09317497e-01  1.57009246e-16 -3.58792680e-01]]


## 4: Compute reduced SVD of $Y$ to obtain $Y=\tilde{U}\tilde{\Sigma}\tilde{V}^T$

In [21]:
U_tilde, Sigma_tilde, VH_tilde = np.linalg.svd(Y, full_matrices=False)
idx = Sigma_tilde > 10e-6
Sigma_tildeInv = np.diag (1/Sigma_tilde[idx])
Sigma_tilde = np.diag (Sigma_tilde[idx])
V_tilde = VH_tilde.T
V_tilde = V_tilde[:,idx]
print (V_tilde)

[[ 8.61705068e-01]
 [ 2.22044605e-16]
 [-5.07409475e-01]]


## 5: $A=V\Sigma^{-1}\tilde{V}$

In [22]:
A_t = V @ SigmaInv @ V_tilde
print (A_t)

[[ 0.30769231]
 [ 0.30769231]
 [ 0.53846154]
 [-0.46153846]]


## 6: Computing objective function for the solution $A$ when $S_2=S_t$

In [23]:
Sb = Hb.T @ Hb
Sw = Hw.T @ Hw
St = Sw + Sb
StY = A_t.T @ St @ A_t
SbY = A_t.T @ Sb @ A_t
obj_st = np.trace(np.linalg.pinv (StY) @ SbY)
print ('Objective Value When S2=St: ', obj_st)

Objective Value When S2=St:  1.0


## Comparing Objective values

In [24]:
print ('Objective Value When S2=Sw: ', obj_sw)
print ('Objective Value When S2=St: ', obj_st)

Objective Value When S2=Sw:  0.13829787234042565
Objective Value When S2=St:  1.0


# It follows that the use of SVD Algorithm with $S_2=S_w$ leads to suboptimal solutions.

# <font color='red'> Another Very Strong Evidence</font>

Another very strong evidence that shows the choice $S_2=S_w$ for the SVD algorithm is wrong, is that the solution of the SVD algorithm with $S_2=S_w$ is not an eigenvector.

In [27]:
(St @ A_w) / (Sb @ A_w)

array([[ 3.53846154],
       [ 1.69230769],
       [-2.46153846],
       [ 1.69230769]])

In [29]:
(St @ A_t) / (Sb @ A_t)

array([[1.],
       [1.],
       [1.],
       [1.]])

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>